**동시성**

- 컴퓨터가 같은 시간에 여러 다른 작업을 처리하는 것처럼 보이게 하는 것

   - CPU 코어가 하나일지라도 여러 프로그램이 번갈아가면서 실행되면 동시에 수행하는 것 같은 착각을 부른다.

- 어떤 특정 유형의 문제를 해결하기 위한 도구

- 여러 다양한 실행 경로나 다양한 I/O 흐름을 제공하여 문제를 해결하는 과정이 동시에 독립적 시행으로 보임
<br>

**병렬성**

- 같은 시간에 여러 다른 작업을 실제로 처리

   - CPU 코어가 여러 개인 컴퓨터에 여러 프로그램을 동시에 실행

# 자식 프로세스를 관리하기 위해서 subprocess를 사용하라

In [1]:
"""
파이썬은 하위 프로세스를 실행하는 방법이 많으므로
자식 프로세스를 관리해야하는데 이때 subprocess 모듈 사용
"""

import subprocess

result = subprocess.run(['echo', '자식 프로세스가 보내는 인사!'],
                        capture_output = True,
                        encoding='utf-8')

result.check_returncode() #예외가 발생하지 않으면 문제없이 잘 종료

print(result.stdout)

자식 프로세스가 보내는 인사!



In [ ]:


"""
파이썬에서 subprocess 등의 모듈을 통해 실행한 자식 프로세스는
부모 프로세스인 파이썬 인터프리터와 독립적으로 실행

run함수 대신 Popen클래스를 사용해 하위 프로세세스 만들면
주기적으로 상태 검사 가능
"""

proc = subprocess.Popen(['sleep', '1'])
while proc.poll() is None:
    print('작업 중...')
    #시간이 걸리는 작업을 여기서 수행

print('종료 상태', proc.poll())

In [ ]:
"""
자식 프로세스와 부모를 분리하면 부모 프로세스가 원하는 개수만큼 많은
자식 프로세스 병렬 실행

시간의 단축화
"""

import time

start = time.time()
sleep_procs = []
for _ in range(10):
    proc = subprocess.Popen(['sleep', '1'])
    sleep_procs.append(proc)

for proc in sleep_procs:
    proc.communicate()
    end = time.time()
    delta = end - start
    print(f'{delta:.3} 초 만에 끝남')

#만약 순차적으로 시행되며 1 * 10인 10초 이상이다.

In [5]:
#파이썬 프로그램의 데이터 파이프를 사용해서 하위 프로세스로 보내거나 하위 프로세스의 출력 받아서 병렬 작업 수행

#명령줄 인자를 사용해서 자식 프로세스를 시작하고 자식 프로세스와 I/O파이프 연결

import os
def run_encrypt(data):
    env = os.environ.copy()
    env['password'] = 'zf7ShyBhZ0raQDdE/FiZpm/m/8f9X+M1'
    proc = subprocess.Popen(
        ['openss1', 'enc', '-des3', '-pass', 'env:password'],
        env = env,
        stdin = subprocess.PIPE,
        stdout = subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush() #자식이 입력을 받도록 보장
    return proc

In [ ]:
#파이프를 통해서 사용자 입력, 파일 핸들, 네트워크 소켓 등에서 받은 데이터를
#암호화 함수에 보내기

procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_encrypt(data)
    procs.append(proc)

In [7]:
#자식 프로세스가 끝나는 것을 기다린 후 마지막 출력 가져오기

for proc in procs:
    out, _ = proc.communicate()
    print(out[-10:])

In [9]:
#openssl 명렬줄 도구로 하위 프로세스 만들기

def run_hash(input_stdin):
    return subprocess.Popen(
        ['openssl','dgst','-whirlpool','-binary'],
        stdin = input_stdin,
        stdout = subprocess.PIPE
    )

In [ ]:
#입력이나 출력 파이프를 기다리면서 block가 두렵다면
#timeout 파라미터를 communicate메서드에 전달
proc = subprocess.Popen(['sleep',  '10'])
try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()

print('종료 상태', proc.poll())



### Summary

- subprocess 모듈을 사용해 자식 프로세스를 실행하고 입력과 출력 스트림을 관리

- 자식 프로세스는 파이썬 인터프리터와 병렬로 실행되므로 CPU 코어를 최대로 씀

- 간단하게 자식 프로세스를 실행하고 싶은 경우에는 run 편의 함수를 사용하고 유닉스 스타일의 파이프라인이 필요하면 Popen 클래스 사용

- 자식 프로세스가 멈추는 경우나 교착 상태를 방직하려면 communicate메서드에 대해서 timeout 파라미터 사용